In [50]:
%pip  install keras
%pip install sklearn
%pip install numpy
%pip install nltk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Text classification using Deep Neural Networks

In [6]:
from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import matthews_corrcoef

Pre-processing

In [12]:
def preprocess_text(text):
    text = text.lower()
    # URL removed
    text = re.sub(r'http\S+|www.\S+', '', text)
    # mention and hashtag removed
    text = re.sub(r'@\S+|#S+', '', text)
    # non-ASCII removed
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # numbers removed
    text = re.sub(r'\d+', '', text)
    # whitespaces removed
    text = re.sub(r'\s+', ' ', text)
    # punctuations removed
    text = re.sub(r'[^\w\s]', '', text)
    # 'rt' / 'retweet' removed
    text = re.sub(r'\b(rt|retweet)\b', '', text.lower())
    # stopwords removed
    #stop_words = set(stopwords.words('english'))
    #text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    # Lemmatization
    #lemmatizer = WordNetLemmatizer()
    #text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

tweets_df = pd.read_csv('twitterData.csv')
tweets_df['cleaned_text'] = tweets_df['text'].apply(preprocess_text)
tweets_df.drop_duplicates(inplace=True)

le = preprocessing.LabelEncoder()
X = tweets_df['cleaned_text'].values.tolist()
y = le.fit_transform(tweets_df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)


In [3]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [13]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 20)
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=10,
                              batch_size=128,
                              verbose=2)

predicted = model_DNN.predict(X_test_tfidf)

print(metrics.classification_report(y_test, np.argmax(predicted, axis=1)))

tf-idf with 13941 features
Epoch 1/10
34/34 - 6s - loss: 2.4121 - accuracy: 0.1418 - val_loss: 2.1604 - val_accuracy: 0.1274 - 6s/epoch - 178ms/step
Epoch 2/10
34/34 - 5s - loss: 2.1237 - accuracy: 0.1522 - val_loss: 2.0089 - val_accuracy: 0.2764 - 5s/epoch - 158ms/step
Epoch 3/10
34/34 - 5s - loss: 1.7543 - accuracy: 0.3067 - val_loss: 1.4753 - val_accuracy: 0.5104 - 5s/epoch - 148ms/step
Epoch 4/10
34/34 - 5s - loss: 0.8682 - accuracy: 0.6831 - val_loss: 0.9867 - val_accuracy: 0.7038 - 5s/epoch - 151ms/step
Epoch 5/10
34/34 - 5s - loss: 0.4290 - accuracy: 0.8487 - val_loss: 0.9522 - val_accuracy: 0.7679 - 5s/epoch - 153ms/step
Epoch 6/10
34/34 - 5s - loss: 0.2316 - accuracy: 0.9212 - val_loss: 1.1225 - val_accuracy: 0.7528 - 5s/epoch - 162ms/step
Epoch 7/10
34/34 - 6s - loss: 0.1502 - accuracy: 0.9467 - val_loss: 0.9955 - val_accuracy: 0.8047 - 6s/epoch - 164ms/step
Epoch 8/10
34/34 - 4s - loss: 0.1105 - accuracy: 0.9667 - val_loss: 0.9528 - val_accuracy: 0.8189 - 4s/epoch - 128ms/st

In [14]:
print("Performance of Deep Neural Network for text classification")
# Accuracy score
print('accuracy is',accuracy_score(y_test,np.argmax(predicted, axis=1)))
mcc = matthews_corrcoef(y_test, np.argmax(predicted, axis=1))
print("MCC:", mcc)
# Macro-Averaging
macro_accuracy = accuracy_score(y_test, np.argmax(predicted, axis=1))
macro_precision, macro_recall, macro_f1, _ = score(y_test, np.argmax(predicted, axis=1), average='macro')
print("Macro-Averaging Metrics:")
print(f"Accuracy: {macro_accuracy:.2f}")
print(f"Precision: {macro_precision:.2f}")
print(f"Recall: {macro_recall:.2f}")
print(f"F1 Score: {macro_f1:.2f}")
# Micro-Averaging
micro_accuracy = accuracy_score(y_test, np.argmax(predicted, axis=1))
micro_precision, micro_recall, micro_f1, _ = score(y_test, np.argmax(predicted, axis=1), average='micro')
print("\nMicro-Averaging Metrics:")
print(f"Accuracy: {micro_accuracy:.2f}")
print(f"Precision: {micro_precision:.2f}")
print(f"Recall: {micro_recall:.2f}")
print(f"F1 Score: {micro_f1:.2f}")

Performance of Deep Neural Network for text classification
accuracy is 0.7830188679245284
MCC: 0.7544188545178457
Macro-Averaging Metrics:
Accuracy: 0.78
Precision: 0.80
Recall: 0.80
F1 Score: 0.80

Micro-Averaging Metrics:
Accuracy: 0.78
Precision: 0.78
Recall: 0.78
F1 Score: 0.78


###without stopwords & lemmetization!!!!

In [48]:
print(metrics.classification_report(y_test, np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       216
           1       0.78      0.86      0.82       181
           2       0.90      0.78      0.84       215
           3       0.91      0.83      0.87       188
           4       0.90      0.90      0.90       196
           5       0.97      0.96      0.96       201
           6       0.84      0.87      0.85       217
           7       0.92      0.90      0.91       192
           8       0.56      0.97      0.71        31
           9       0.80      0.95      0.87        21

    accuracy                           0.89      1658
   macro avg       0.86      0.90      0.87      1658
weighted avg       0.90      0.89      0.89      1658



Text classification using RNN

In [15]:
from keras.layers import Dropout, Dense, GRU, Embedding
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.datasets import fetch_20newsgroups

In [16]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open(".\\glove.twitter.27B.50d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

def Build_Model_RNN_Text(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    """
    def buildModel_RNN(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    word_index in word index ,
    embeddings_index is embeddings index, look at data_helper.py
    nClasses is number of classes,
    MAX_SEQUENCE_LENGTH is maximum lenght of text sequences
    """

    model = Sequential()
    hidden_layer = 3
    gru_node = 32

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))


    print(gru_node)
    for i in range(0,hidden_layer):
        model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
        model.add(Dropout(dropout))
    model.add(GRU(gru_node, recurrent_dropout=0.2))
    model.add(Dropout(dropout))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(nclasses, activation='softmax'))


    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

In [18]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_RNN = Build_Model_RNN_Text(word_index,embeddings_index, 20)

model_RNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=10,
                              batch_size=128,
                              verbose=2)

predicted = model_RNN.predict(X_test_Glove)

print(metrics.classification_report(y_test, np.argmax(predicted, axis=1)))

Found 15987 unique tokens.
(5298, 500)
Total 1193514 word vectors.
32
Epoch 1/10
34/34 - 114s - loss: 2.5342 - accuracy: 0.1402 - val_loss: 2.2034 - val_accuracy: 0.1500 - 114s/epoch - 3s/step
Epoch 2/10
34/34 - 92s - loss: 2.1918 - accuracy: 0.1498 - val_loss: 2.1173 - val_accuracy: 0.2028 - 92s/epoch - 3s/step
Epoch 3/10
34/34 - 92s - loss: 2.0629 - accuracy: 0.2006 - val_loss: 1.9480 - val_accuracy: 0.2406 - 92s/epoch - 3s/step
Epoch 4/10
34/34 - 113s - loss: 1.8947 - accuracy: 0.2529 - val_loss: 1.8365 - val_accuracy: 0.2632 - 113s/epoch - 3s/step
Epoch 5/10
34/34 - 120s - loss: 1.7545 - accuracy: 0.3134 - val_loss: 1.7259 - val_accuracy: 0.3491 - 120s/epoch - 4s/step
Epoch 6/10
34/34 - 118s - loss: 1.6058 - accuracy: 0.3952 - val_loss: 1.5728 - val_accuracy: 0.4358 - 118s/epoch - 3s/step
Epoch 7/10
34/34 - 117s - loss: 1.4491 - accuracy: 0.4571 - val_loss: 1.4312 - val_accuracy: 0.5038 - 117s/epoch - 3s/step
Epoch 8/10
34/34 - 116s - loss: 1.2818 - accuracy: 0.5415 - val_loss: 1.3

In [20]:
print("Performance of Recurrent Neural Network for text classification")
print(metrics.classification_report(y_test, np.argmax(predicted, axis=1)))

Performance of Recurrent Neural Network for text classification
              precision    recall  f1-score   support

           0       0.74      0.70      0.72        69
           1       0.58      0.73      0.65       171
           2       0.78      0.54      0.64       135
           3       0.53      0.65      0.58       124
           4       0.84      0.76      0.80       122
           5       0.65      0.60      0.62        84
           6       0.62      0.76      0.68       156
           7       0.76      0.72      0.74       151
           8       0.50      0.22      0.30        23
           9       0.80      0.16      0.27        25

    accuracy                           0.67      1060
   macro avg       0.68      0.58      0.60      1060
weighted avg       0.68      0.67      0.66      1060



In [21]:
print("Performance of Recurrent Neural Network for text classification")
# Accuracy score
print('accuracy is',accuracy_score(y_test,np.argmax(predicted, axis=1)))
mcc = matthews_corrcoef(y_test, np.argmax(predicted, axis=1))
print("MCC:", mcc)
# Macro-Averaging
macro_accuracy = accuracy_score(y_test, np.argmax(predicted, axis=1))
macro_precision, macro_recall, macro_f1, _ = score(y_test, np.argmax(predicted, axis=1), average='macro')
print("Macro-Averaging Metrics:")
print(f"Accuracy: {macro_accuracy:.2f}")
print(f"Precision: {macro_precision:.2f}")
print(f"Recall: {macro_recall:.2f}")
print(f"F1 Score: {macro_f1:.2f}")
# Micro-Averaging
micro_accuracy = accuracy_score(y_test, np.argmax(predicted, axis=1))
micro_precision, micro_recall, micro_f1, _ = score(y_test, np.argmax(predicted, axis=1), average='micro')
print("\nMicro-Averaging Metrics:")
print(f"Accuracy: {micro_accuracy:.2f}")
print(f"Precision: {micro_precision:.2f}")
print(f"Recall: {micro_recall:.2f}")
print(f"F1 Score: {micro_f1:.2f}")

Performance of Recurrent Neural Network for text classification
accuracy is 0.6650943396226415
MCC: 0.6178581905583603
Macro-Averaging Metrics:
Accuracy: 0.67
Precision: 0.68
Recall: 0.58
F1 Score: 0.60

Micro-Averaging Metrics:
Accuracy: 0.67
Precision: 0.67
Recall: 0.67
F1 Score: 0.67


Convolutional Neural Networks

In [22]:
from keras.layers import Dropout, Dense,Input,Embedding,Flatten, MaxPooling1D, Conv1D
from keras.models import Sequential,Model
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate

def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open(".\\glove.twitter.27B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

def Build_Model_CNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):

    """
        def buildModel_CNN(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
        word_index in word index ,
        embeddings_index is embeddings index, look at data_helper.py
        nClasses is number of classes,
        MAX_SEQUENCE_LENGTH is maximum lenght of text sequences,
        EMBEDDING_DIM is an int value for dimention of word embedding look at data_helper.py
    """

    model = Sequential()
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)

    # applying a more complex convolutional approach
    convs = []
    filter_sizes = []
    layer = 5
    print("Filter  ",layer)
    for fl in range(0,layer):
        filter_sizes.append((fl+2))

    node = 128
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    for fsz in filter_sizes:
        l_conv = Conv1D(node, kernel_size=fsz, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(5)(l_conv)
        #l_pool = Dropout(0.25)(l_pool)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)
    l_cov1 = Conv1D(node, 5, activation='relu')(l_merge)
    l_cov1 = Dropout(dropout)(l_cov1)
    l_pool1 = MaxPooling1D(5)(l_cov1)
    l_cov2 = Conv1D(node, 5, activation='relu')(l_pool1)
    l_cov2 = Dropout(dropout)(l_cov2)
    l_pool2 = MaxPooling1D(30)(l_cov2)
    l_flat = Flatten()(l_pool2)
    l_dense = Dense(1024, activation='relu')(l_flat)
    l_dense = Dropout(dropout)(l_dense)
    l_dense = Dense(512, activation='relu')(l_dense)
    l_dense = Dropout(dropout)(l_dense)
    preds = Dense(nclasses, activation='softmax')(l_dense)
    model = Model(sequence_input, preds)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [23]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)

model_CNN = Build_Model_CNN_Text(word_index,embeddings_index, 20)
model_CNN.summary()
model_CNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

predicted = model_CNN.predict(X_test_Glove)
predicted = np.argmax(predicted, axis=1)

Found 15987 unique tokens.
(5298, 500)
Total 1193514 word vectors.
Filter   5
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 500, 50)      799400      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 499, 128)     12928       ['embedding_1[0][0]']            
                                                                                                  
 conv1d_1 (Conv1D)              (None, 498, 128)     19328       ['embedding_1[0][0]']            
                

In [25]:
print("Performance of Convolutional Neural Network for text classification")
print(metrics.classification_report(y_test, predicted))

Performance of Convolutional Neural Network for text classification
              precision    recall  f1-score   support

           0       0.93      0.81      0.87        69
           1       0.71      0.80      0.75       171
           2       0.68      0.72      0.70       135
           3       0.84      0.75      0.79       124
           4       0.81      0.79      0.80       122
           5       0.79      0.79      0.79        84
           6       0.69      0.74      0.72       156
           7       0.88      0.77      0.82       151
           8       0.61      0.83      0.70        23
           9       0.81      0.68      0.74        25

    accuracy                           0.77      1060
   macro avg       0.78      0.77      0.77      1060
weighted avg       0.78      0.77      0.77      1060



In [28]:
print("Performance of Convolutional Neural Network for text classification")
# Accuracy score
print('accuracy is',accuracy_score(y_test,predicted))
mcc = matthews_corrcoef(y_test, predicted)
print("MCC:", mcc)
# Macro-Averaging
macro_accuracy = accuracy_score(y_test, predicted)
macro_precision, macro_recall, macro_f1, _ = score(y_test, predicted, average='macro')
print("Macro-Averaging Metrics:")
print(f"Accuracy: {macro_accuracy:.2f}")
print(f"Precision: {macro_precision:.2f}")
print(f"Recall: {macro_recall:.2f}")
print(f"F1 Score: {macro_f1:.2f}")
# Micro-Averaging
micro_accuracy = accuracy_score(y_test, predicted)
micro_precision, micro_recall, micro_f1, _ = score(y_test, predicted, average='micro')
print("\nMicro-Averaging Metrics:")
print(f"Accuracy: {micro_accuracy:.2f}")
print(f"Precision: {micro_precision:.2f}")
print(f"Recall: {micro_recall:.2f}")
print(f"F1 Score: {micro_f1:.2f}")

Performance of Convolutional Neural Network for text classification
accuracy is 0.7669811320754717
MCC: 0.7346007746743662
Macro-Averaging Metrics:
Accuracy: 0.77
Precision: 0.78
Recall: 0.77
F1 Score: 0.77

Micro-Averaging Metrics:
Accuracy: 0.77
Precision: 0.77
Recall: 0.77
F1 Score: 0.77


Recurrent Convolutional Neural Networks

In [29]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open(".\\glove.twitter.27B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

def Build_Model_RCNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50):

    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector



    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [30]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)

model_RCNN = Build_Model_CNN_Text(word_index,embeddings_index, 20)
model_RCNN.summary()
model_RCNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=15,
                              batch_size=128,
                              verbose=2)

predicted = model_RCNN.predict(X_test_Glove)
predicted = np.argmax(predicted, axis=1)

Found 15987 unique tokens.
(5298, 500)
Total 1193514 word vectors.
Filter   5
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 500, 50)      799400      ['input_2[0][0]']                
                                                                                                  
 conv1d_7 (Conv1D)              (None, 499, 128)     12928       ['embedding_2[0][0]']            
                                                                                                  
 conv1d_8 (Conv1D)              (None, 498, 128)     19328       ['embedding_2[0][0]']            
              

In [31]:
print("Performance of Recurrent Convolutional Neural Network for text classification")
print(metrics.classification_report(y_test, predicted))

Performance of Recurrent Convolutional Neural Network for text classification
              precision    recall  f1-score   support

           0       0.90      0.78      0.84        69
           1       0.62      0.84      0.71       171
           2       0.75      0.64      0.69       135
           3       0.84      0.69      0.76       124
           4       0.90      0.75      0.82       122
           5       0.59      0.77      0.67        84
           6       0.69      0.74      0.72       156
           7       0.88      0.77      0.82       151
           8       0.64      0.61      0.62        23
           9       0.82      0.56      0.67        25

    accuracy                           0.74      1060
   macro avg       0.76      0.72      0.73      1060
weighted avg       0.76      0.74      0.74      1060



In [32]:
print("Performance of Recurrent Convolutional Neural Network for text classification")
# Accuracy score
print('accuracy is',accuracy_score(y_test,predicted))
mcc = matthews_corrcoef(y_test, predicted)
print("MCC:", mcc)
# Macro-Averaging
macro_accuracy = accuracy_score(y_test, predicted)
macro_precision, macro_recall, macro_f1, _ = score(y_test, predicted, average='macro')
print("Macro-Averaging Metrics:")
print(f"Accuracy: {macro_accuracy:.2f}")
print(f"Precision: {macro_precision:.2f}")
print(f"Recall: {macro_recall:.2f}")
print(f"F1 Score: {macro_f1:.2f}")
# Micro-Averaging
micro_accuracy = accuracy_score(y_test, predicted)
micro_precision, micro_recall, micro_f1, _ = score(y_test, predicted, average='micro')
print("\nMicro-Averaging Metrics:")
print(f"Accuracy: {micro_accuracy:.2f}")
print(f"Precision: {micro_precision:.2f}")
print(f"Recall: {micro_recall:.2f}")
print(f"F1 Score: {micro_f1:.2f}")

Performance of Recurrent Convolutional Neural Network for text classification
accuracy is 0.7415094339622641
MCC: 0.7067780882248423
Macro-Averaging Metrics:
Accuracy: 0.74
Precision: 0.76
Recall: 0.72
F1 Score: 0.73

Micro-Averaging Metrics:
Accuracy: 0.74
Precision: 0.74
Recall: 0.74
F1 Score: 0.74
